In [14]:
# Fourier, Characteritic function, clabiration of Heston model

In [35]:
# Import modules 
import math
import numpy as np
from scipy.stats import norm
import scipy.optimize as optimize
import matplotlib.pyplot as plt

In [36]:
# Market information, parameters, we want to find values for kappa, theta, rho...?
# Market, option,share dynamics and information
r     = 0.06 # continously compounded risk free rate
S_0   = 100  # initial stock price

# Stochastic volatility parameters
v_0   = 0.06
#kappa = None, values must be estimated
#theta = None, values must be estimated
#sigma = None, value must be estimated
rho   = -0.4

# Market option specific information
K     = np.array([110,100,90])  # Strike price of call option
price = [8.02,12.63,18.72]
k_log = np.log(K)
k_log.shape = (3,1)
T     = 1   # Time until maturity 

#Approximation information
# Initialisin and calculating parameters for estimation t_max, N, t_delta for Gil Pelaez
t_max    = 30
N        = 100
delta_t  = t_max/N
# The array of various t_k points
from_1_N = np.linspace(1,N,N)
t_k      = (from_1_N-1/2)*delta_t  

In [40]:
# Characteristic function code
# Characteristic functon of sT = log(ST) => exp(C(T;u) + D(T,u)*v_t + iu*log(S_t))  # i complex number
# C(T,u) = riTu + theta*x[Tx - 1/a*log(1-gexp(d*r)/1-g)]
# D(T,u) = [(1-exp(d*r))/1-gexp(d*r)]*x_
#  x_    = b-d/2a
# x+     = b+d/2a
# c      = (-u**2+ui)/2
# g      = x_/x+
# d      = sqrt(b**2-4ac)
# b      = kappa - rho*sigma*i*u
# a      = sigma**2/2

def a(sigma):
    return sigma**2/2

def b(u, theta, kappa, sigma):
    return kappa - rho*sigma*1j*u

def c(u, theta, kappa, sigma):
    return -(u**2 + u*1j)/2

def d(u, theta, kappa, sigma):
    return np.sqrt(b(u,theta,kappa,sigma)**2 - 4*a(sigma)*c(u, theta, kappa, sigma))

def x_minus(u, theta, kappa, sigma):
    return (b(u, theta, kappa, sigma)-d(u, theta, kappa, sigma))/(2*a(sigma))

def x_plus(u, theta, kappa, sigma):
    return (b(u,theta, kappa, sigma)+d(u, theta, kappa, sigma))/(2*a(sigma))

def g(u, theta, kappa, sigma):
    return x_minus(u,theta, kappa, sigma)/x_plus(u, theta, kappa, sigma)

def C(u, theta, kappa, sigma):
    value = T*x_minus(u,theta, kappa, sigma) - np.log((1-g(u,theta,kappa,sigma)*np.exp(d(u,theta,kappa,sigma)*-T))/(1-g(u,theta,kappa,sigma)))/a(sigma)
    return r*1j*T*u + theta*kappa*value

def D(u, theta, kappa, sigma):
    value1 = 1-np.exp(d(u,theta,kappa,sigma)*-T)
    value2 = 1-g(u,theta,kappa,sigma)*np.exp(d(u,theta,kappa,sigma)*-T)
    return (value1/value2)*x_minus(u,theta,kappa,sigma)

def log_char(u, theta, kappa,sigma):  #char_M1
    return np.exp(C(u,theta,kappa,sigma) + D(u,theta,kappa,sigma)*v_0 + 1j*u*np.log(S_0))
    
# Change of measure, converting charactersitic function directly instead of direct change of measure
# Risk neutral measure under Q_S so we can apply Gil Pelaez
# char_S(u) = char(u-i) / char(-i)
    
def adj_char(u, theta, kappa, sigma):    #char_M2
    return log_char(u-1j,theta,kappa,sigma) / log_char(-1j, theta, kappa, sigma)   

In [48]:
# Calibration functions
def heston_pricer(array_estimates):
    theta = array_estimates[0]
    kappa    = array_estimates[1]
    sigma = array_estimates[2]
    first_intergral   = np.sum((((np.exp(-1j*t_k*k_log))*adj_char(t_k,theta,kappa,sigma)).imag/t_k)*delta_t,axis=1)
    second_intergral  = np.sum((((np.exp(-1j*t_k*k_log))*log_char(t_k,theta,kappa,sigma)).imag/t_k)*delta_t,axis=1)
    return S_0*(1/2 + first_intergral/np.pi) - np.exp(-r*T)*K*(1/2 + second_intergral/np.pi)


def optimizer(array_estimates):    
    return  sum(np.abs(price- heston_pricer(array_estimates)))

In [49]:
# Find optimal values using initial estimates
optimal_values = optimize.fmin_slsqp(optimizer,(0.1,3,0.1))

C:\Users\Zed\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: overflow encountered in exp
C:\Users\Zed\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Zed\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys
C:\Users\Zed\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0024517836240907087
            Iterations: 18
            Function evaluations: 144
            Gradient evaluations: 18


C:\Users\Zed\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in cdouble_scalars
C:\Users\Zed\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in cdouble_scalars
C:\Users\Zed\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in cdouble_scalars
C:\Users\Zed\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in log
C:\Users\Zed\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in cdouble_scalars
C:\Users\Zed\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in cdouble_scalars


In [51]:
optimal_values

array([0.05988376, 3.07129354, 0.25690196])